# `numpy`中`ndarray`对象的构造, 处理与应用

In [1]:
#以别名形式导入numpy. 
import numpy as np; 
#导入与ndarray有关的数据类型. 
from numpy import int8, int16, int32, int64; 
from numpy import uint8, uint16, uint32, uint64; 
from numpy import float16, float32, float64; 
from numpy import complex64, complex128; 

## `numpy.ndarray`对象的特征
与Python中内置的`list`和`tuple`对象不同, `numpy.array`对象中的结构和其中的每个元素必须满足以下要求: 

* 所有元素必须是**同一个类**的不同实例; 
> 相较之下, 同一个`list`或`tuple`对象中, 不同类型(如`int`和`str`)的元素可以共存. 
* 所有元素所在的类是**定长**的, 因此不同实例在内存中占用的大小是固定的; 
    * 如果**强制使用具有变长类型**实例的`list`或者`tuple`构造`ndarray`, 此时**如未指定`dtype`, 则`dtype`将被设定为`object`**, `ndarray`将在数组中存储变长对象的地址, *成为一个指针数组*. 
* 当`array`对象是多维数组(数组嵌套数组)时, 在任何一级数组中, 被嵌套的数组具有相同的长度. 

因此, `numpy.array`的数据部分, 在内存中可能以顺序结构存储

* 使用下标访问时, 可直接根据下标, 确定目标元素地址相对于基址的偏移量. 
* 功能上相当于部分早期编程语言中(如C, Basic, Pascal, IDL)的`array`, **只能存储同类型数据**; 
* 在大批量检索和修改个别数据的场合, 其速度比python中内置的`list`更快. 

## `numpy.ndarray`对象的构造

### 将`list`或`tuple`转换为`numpy.ndarray`
用法: 
```python
numpy.array(seq, dtype = dt)
```
* `seq` 列表(`list`对象)或元组(`tuple`对象)
* `seq`结构要求: 
    * 当`seq`存在嵌套的`list`或`tuple`时, `seq`的所有元素必须等长; 
    * 当前述嵌套出现二层以上时, 要求嵌套的每层元素均满足上一要求; 
* `dtype = dt` 数组内各元素的类型和字长
* 支持的类型

|表示范围|类型名称(需使用`from numpy import ...`<br>导入为当前上下文中的名称)|
|:-:|:-|
|有符号整数|`int8`, `int16`, `int32`, `int64`|
|无符号整数|`uint8`, `uint16`, `uint32`, `uint64`|
|浮点小数|`float16`, `float32`, `float64`|
|复数|`complex64`, `complex128`|

> 由于字长固定, 有符号整数和无符号整数在执行大数运算时可能发生溢出, 溢出时**不会报错**, 会将溢出后的**高位截断**, 得到不符合预期的结果. 例如
>
> |计算语句|计算结果|
> |:-|:-|
> |`uint8(255) + uint8(1)`|`uint8(0)`|
> |`int8(42) * int8(21)`|`int8(114)`|
> |`int8(42) * int8(42) // int8(2)`|`int8(-14)`|

In [2]:
#定义二维旋转变换矩阵(绕原点旋转)
def rot2D_matrix(rot_angle): 
    cθ = np.cos(rot_angle); sθ = np.sin(rot_angle); 
    #使用齐次形式的坐标, 因此二维旋转变换表示为3×3方阵
    ls = [[cθ, -sθ, 0], [sθ, cθ, 0], [0, 0, 1]]; 
    #将中间变量(list对象)转换为ndarray对象, 作为函数返回值
    return(np.array(ls, dtype = float64)); 

### 利用`numpy`函数构造特殊数组
`numpy`中可利用特定函数构造满足一定性质的`ndarray`. 
* 这些数组被视为相应的$n$阶张量时, 具有特殊的数学意义. 
* 所有方法均返回`np.ndarray`对象, 支持`dtype = dt`选项. 

|数组类型|方法|备注|
|:-|:-|:-|
|生成元素全零的$n$阶张量|`np.zeros(dim)`|张量乘法的零元. <br>`dim`为`tuple`对象, 指定$n$阶张量中<br>第$i$阶($1 \le i \le n$)的维数为$dim[i-1]$|
|生成$n$阶单位矩阵<br>(主对角线元素均为1, 其余位置元素均为0)|`np.identity(n)`|矩阵乘法的左(或右)单位元. <br>`n`为`int`对象, 指定矩阵的行数和<br>列数均为$n$|
|生成元素全为$x$的$n$阶张量|`np.full(dim, x)`|`dim`为`tuple`对象, 指定$n$阶张量中<br>第$i$阶($1 \le i \le n$)的维数为$dim[i-1]$<br>调用本方法时, 先计算`x`的值, 再利用<br>`x`的计算结果构造数组|
|生成以$a$为首项, $d$为公差, 末项小于$b$<br>($d<0$)或大于$b$($d>0$)的等差数组|`np.arange(a, b, d)`|功能上等效于<br>`np.array(range(a, b, d))`|
|在$a$(含)和$b$(含)之间以均匀间隔采样, <br>生成长度为$n$的等差数组|`np.linspace(a, b, n)`|`np.linspace`不能用于构造线性空间|
|在$a$(含)和$b$(含)之间以非均匀间隔采样, <br>生成长度为$n$的等比数组|`np.geomspace(a, b, n)`||

In [3]:
#比较不同的数列生成方法的差异
for seqgen in "arange", "linspace", "geomspace": 
    seq = "np." + seqgen + "(4, 64, 9)"; 
    print(f'{seq} -> {eval(seq[: -1] + ", dtype = float32)")}'); 

np.arange(4, 64, 9) -> [ 4. 13. 22. 31. 40. 49. 58.]
np.linspace(4, 64, 9) -> [ 4.  11.5 19.  26.5 34.  41.5 49.  56.5 64. ]
np.geomspace(4, 64, 9) -> [ 4.        5.656854  8.       11.313708 16.       22.627417 32.
 45.254833 64.      ]


## `numpy.ndarray`对象的主要属性
|属性名称|含义|
|:-|:-|
|`ndim`|数组的阶数, 相当于`len(shape)`|
|`size`|数组中包含元素的总数, 相当于`np.product(shape)`|
|`shape`|`tuple`对象, 其中`size[i]`($0 \le i \le ndim - 1$)为数组第($i+1$)阶的维数|
|`dtype`|数组中元素的类型|

In [4]:
#二维平面内逆时针旋转60°表示成3×3方阵, 是一个2阶张量. 
matA = rot2D_matrix(np.pi / 3); 
print(matA.ndim, matA.size, matA.shape, matA.dtype); 

2 9 (3, 3) float64


## `numpy.ndarray`对象的访问和编辑

### 用索引(簇)访问`numpy.ndarray`的内容
采用索引, 索引簇访问`ndarray`中个别元素或者子数组的方法, 与访问`list`对象中个别元素的方法相似

* `ndarray`**可以直接将多级索引顺次列出**(相当于构建了一个`tuple`)**后, 括注在同一个方括号下**
> 
> 例如, 对二阶`ndarray`中上起第2行, 左起第2列元素, 采用下述语句索引是合法的: 
> ```python
> np.array([[1, 0], [0, 2]]) [1, 1]
> ```
> `list`不支持这一索引方式. 对相同结构的`list`中相同位置的元素, 采用这一方式将报错: 
> ```
> TypeError: list indices must be integers or slices, not tuple
> ```
> 嵌套`list`的多级索引语句只能写作: 
> ```python
> list([[1, 0], [0, 2]]) [1][1]
> ```
> 可能原因: 这两类对象中元素在内存中的存储结构不同. 

* 返回的`ndarray`的**`ndim`属性值**, 等于访问时**使用的索引簇**的个数. 

值得注意的是: 数组的`size`属性为`(n, )`, `(1, n)`和`(n, 1)`时, 数组所表达意义的差异: 
* 当`numpy.ndarray.size`为`(n, )`(一元`tuple`)时, 数组表示一阶$n$维向量; 在点乘运算中, 根据实际情况, 既有可能被解释为行向量, 也有可能被解释为列向量; 
* 当`numpy.ndarray.size`为`(1, n)`, `(n, 1)`时, 数组分别表示二阶张量(矩阵)中的$n$维行向量, 列向量; 只有通过转置运算, 才能在上述两种形态间转化. 

In [5]:
for idx in "", "[: , 1]", "[: , 1: 2]", "[1: 2, : ]": 
    tns = eval("matA" + idx); 
    print(f'matA{idx} -> {tns.shape}')

matA -> (3, 3)
matA[: , 1] -> (3,)
matA[: , 1: 2] -> (3, 1)
matA[1: 2, : ] -> (1, 3)


### 通过构造`generator`, `enumerate`或`zip`对象遍历`ndarray`
通过构造`generator`, `enumerate`或`zip`对象遍历访问`ndarray`中子数组的方法, 与访问`list`对象中个别元素的方法相似, 只能作用于`ndarray`的最外层. 

In [6]:
print([row for row in matA])

[array([ 0.5      , -0.8660254,  0.       ]), array([0.8660254, 0.5      , 0.       ]), array([0., 0., 1.])]
